In [2]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder


In [3]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import keras
from keras.layers import Dense

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

In [1]:
from augmentdata import data_augment

Using TensorFlow backend.


In [ ]:
# ! pip install augmentdata

In [5]:
def check_cm_others(y_actual,y_predict):
    from sklearn.metrics import confusion_matrix

    cm1 = confusion_matrix(y_actual,y_predict)
    print('Confusion Matrix : \n', cm1)

    total1=sum(sum(cm1))
    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    tn=cm1[0,0]
    fp=cm1[0,1]
    fn=cm1[1,0]
    tp=cm1[1,1]

    sensitivity1 = tp/(tp+fn)
    print('Sensitivity : ', sensitivity1 )

    specificity1 = tn/(tn+fp)
    print('Specificity : ', specificity1)
    recall=sensitivity1
    precision=tp/(tp+fp)

    print("Precision = ",precision)
    print("Recall = ",recall)

    f1_score=2*(precision*recall)/(precision+recall)

    print("F1 score = ",f1_score)

    return sensitivity1,specificity1,f1_score

def create_model(weight_path,input_dim):
    
    checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model=None
    model = Sequential()
    model.add(Dense(20, input_dim=input_dim, 
                activation='relu')) 

    model.add(Dense(20, 
                activation='relu'))
    model.add(Dense(20, 
                activation='relu'))

    model.add(Dense(20, 
                activation='relu'))

    model.add(Dense(20, 
                activation='relu'))
    model.add(Dense(20, 
                activation='relu'))

    model.add(Dense(2, 
                activation='softmax'))
    opt=keras.optimizers.Adam(lr=.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)



    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model, callbacks_list
    

### Data pre processing

In [6]:
df=pd.read_csv("data/Data1.csv")

In [7]:
df.shape

(6000, 45)

In [8]:
df["Case_Control"].value_counts()

Control    4000
Case       2000
Name: Case_Control, dtype: int64

In [9]:
df=df.fillna(0)

In [10]:
for index,row in df.iterrows():
    if row["Case_Control"] == "Case":
        df.at[index,'Case_Control'] = 1
    elif row["Case_Control"] == "Control":
        df.at[index,'Case_Control'] = 0

In [11]:
df['Case_Control'] = df['Case_Control'].astype(int)    

In [12]:
df['Case_Control'].value_counts()

0    4000
1    2000
Name: Case_Control, dtype: int64

In [13]:
label_encoder = LabelEncoder()

df['Nationality'] = df['Nationality'].astype(str)
df.Nationality = label_encoder.fit_transform(df["Nationality"])


df['Gender'] = df['Gender'].astype(str)
df.Gender = label_encoder.fit_transform(df["Gender"])





df['OccupationActivity'] = df['OccupationActivity'].astype(str)
df.OccupationActivity = label_encoder.fit_transform(df["OccupationActivity"])



df['TimeSittingPerDay_Weekend_UsingComputer'] = df['TimeSittingPerDay_Weekend_UsingComputer'].astype(str)
df.TimeSittingPerDay_Weekend_UsingComputer = label_encoder.fit_transform(df["TimeSittingPerDay_Weekend_UsingComputer"])


df['HeavySportActivities_Last7Days_AtLeast10Min'] = df['HeavySportActivities_Last7Days_AtLeast10Min'].astype(str)
df.HeavySportActivities_Last7Days_AtLeast10Min = label_encoder.fit_transform(df["HeavySportActivities_Last7Days_AtLeast10Min"])

df['TimeWalked'] = df['TimeWalked'].astype(str)

df.TimeWalked = label_encoder.fit_transform(df["TimeWalked"])

df['TypicalSleepHours_LastYear'] = df['TypicalSleepHours_LastYear'].astype(str)

df.TypicalSleepHours_LastYear = label_encoder.fit_transform(df["TypicalSleepHours_LastYear"])



df['NapDuringDay_LastYear'] = df['NapDuringDay_LastYear'].astype(str)
df.NapDuringDay_LastYear = label_encoder.fit_transform(df["NapDuringDay_LastYear"])

df['Snoar'] = df['Snoar'].astype(str)
df.Snoar = label_encoder.fit_transform(df["Snoar"])

df['Smoke'] = df['Smoke'].astype(str)
df.Smoke = label_encoder.fit_transform(df["Smoke"])

df['Age40_SmokingRate'] = df['Age40_SmokingRate'].astype(str)
df.Age40_SmokingRate = label_encoder.fit_transform(df["Age40_SmokingRate"])

df['OtherHouseholdSmokers'] = df['OtherHouseholdSmokers'].astype(str)
df.OtherHouseholdSmokers = label_encoder.fit_transform(df["OtherHouseholdSmokers"])

df['Shisha_Smoked'] = df['Shisha_Smoked'].astype(str)
df.Shisha_Smoked = label_encoder.fit_transform(df["Shisha_Smoked"])



df['ParentsSmokedAtYourBirth'] = df['ParentsSmokedAtYourBirth'].astype(str)
df.ParentsSmokedAtYourBirth = label_encoder.fit_transform(df["ParentsSmokedAtYourBirth"])


df['FatherAlive'] = df['FatherAlive'].astype(str)
df.FatherAlive = label_encoder.fit_transform(df["FatherAlive"])


df['HIPWAIST_OUT_MEASURE_TAKEN'] = df['HIPWAIST_OUT_MEASURE_TAKEN'].astype(str)
df.HIPWAIST_OUT_MEASURE_TAKEN = label_encoder.fit_transform(df["HIPWAIST_OUT_MEASURE_TAKEN"])


df['BP_OUT_ARM'] = df['BP_OUT_ARM'].astype(str)
df.BP_OUT_ARM = label_encoder.fit_transform(df["BP_OUT_ARM"])


df['BP_OUT_MANUAL_ENRTY_YN_3'] = df['BP_OUT_MANUAL_ENRTY_YN_3'].astype(str)
df.BP_OUT_MANUAL_ENRTY_YN_3 = label_encoder.fit_transform(df["BP_OUT_MANUAL_ENRTY_YN_3"])


df['BP_OUT_MANUAL_ENRTY_YN_2'] = df['BP_OUT_MANUAL_ENRTY_YN_2'].astype(str)
df.BP_OUT_MANUAL_ENRTY_YN_2 = label_encoder.fit_transform(df["BP_OUT_MANUAL_ENRTY_YN_2"])


df['BP_OUT_ARMBAND_SIZE_USED'] = df['BP_OUT_ARMBAND_SIZE_USED'].astype(str)
df.BP_OUT_ARMBAND_SIZE_USED = label_encoder.fit_transform(df["BP_OUT_ARMBAND_SIZE_USED"])

df['BP_OUT_MANUAL_ENRTY_YN_1'] = df['BP_OUT_MANUAL_ENRTY_YN_1'].astype(str)
df.BP_OUT_MANUAL_ENRTY_YN_1 = label_encoder.fit_transform(df["BP_OUT_MANUAL_ENRTY_YN_1"])


df['BP_OUT_SLEEVE_TYPE'] = df['BP_OUT_SLEEVE_TYPE'].astype(str)
df.BP_OUT_SLEEVE_TYPE = label_encoder.fit_transform(df["BP_OUT_SLEEVE_TYPE"])




In [14]:
# df[["Dummy ID","OccupationActivity"]].head(50)
categorical_columns=["OccupationActivity","TimeSittingPerDay_Weekend_UsingComputer",
                         "TypicalSleepHours_LastYear","NapDuringDay_LastYear","Snoar","OtherHouseholdSmokers",
                         "FatherAlive"
                         ]

# following columns after feature importance
continuos_columns=["Age","HIPWAIST_OUT_WAIST_SIZE","HIPWAIST_OUT_HIPS_SIZE",
                   "BP_OUT_DIASTOLIC_BP_2","BP_OUT_DIASTOLIC_BP_1","BP_OUT_SYSTOLIC_BP_1",
                   "BMI"]
label_column=["Case_Control"]                   
all_useful_columns=categorical_columns+continuos_columns+label_column

df[all_useful_columns].head(12)

,OccupationActivity,TimeSittingPerDay_Weekend_UsingComputer,TypicalSleepHours_LastYear,NapDuringDay_LastYear,Snoar,OtherHouseholdSmokers,FatherAlive,Age,HIPWAIST_OUT_WAIST_SIZE,HIPWAIST_OUT_HIPS_SIZE,BP_OUT_DIASTOLIC_BP_2,BP_OUT_DIASTOLIC_BP_1,BP_OUT_SYSTOLIC_BP_1,BMI,Case_Control
0,0,5,3,1,3,3,2,52,115.0,118.0,84.0,87,139,37.3,1
1,1,1,4,5,1,1,2,55,118.0,117.0,76.0,81,136,37.1,1
2,2,5,3,3,1,3,2,80,82.0,99.0,69.0,58,187,27.1,1
3,3,4,2,3,1,4,3,28,91.0,114.0,88.0,76,130,33.8,1
4,0,1,3,3,3,3,2,51,90.0,124.0,78.0,73,119,35.9,1
5,3,5,2,5,3,3,3,44,91.0,101.0,68.0,65,102,26.7,1
6,0,5,3,2,3,1,2,56,98.0,108.0,59.0,64,112,32.9,1
7,0,5,2,2,3,4,2,55,99.0,104.0,70.0,73,115,26.6,1
8,0,5,3,5,1,1,3,40,98.0,112.0,83.0,82,113,35.1,1
9,0,5,4,1,3,3,2,56,110.0,105.0,77.0,79,138,33.3,1


In [15]:
cols_with_na_500=["HeavySportActivities_Last7Days_AtLeast10Min","TimeWalked",
                  "DaysPerWeek_Walk","TimePerDay_Walk",
                  "Minutes_Walk","Age40_SmokingRate",
                  "HIPWAIST_OUT_MEASURE_TAKEN","BP_OUT_SYSTOLIC_BP_3",
                  "BP_OUT_PULSE_RATE_3","BP_OUT_MANUAL_ENRTY_YN_3",
                  "BP_OUT_DIASTOLIC_BP_3","Smoke","Shisha_Smoked"]




In [16]:
df=df.drop(columns=cols_with_na_500,axis=1)

In [17]:
# keep interesting columns only

df=df.drop(['Dummy ID'], axis=1)
df=df.drop(columns=["HBA 1C %"],axis=1)
df=df.drop(columns=["Nationality"],axis=1)


In [18]:
for col in df.columns:
    df[col]= df[col].astype(float)


In [19]:
# here take 20% out for testing
np.random.seed(42)
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [20]:
print(len(test))
print(len(train))
print(test.shape)
print(train.shape)

1199
4801
(1199, 29)
(4801, 29)


In [21]:
print(test["Case_Control"].value_counts())
print(train["Case_Control"].value_counts())


0.0    792
1.0    407
Name: Case_Control, dtype: int64
0.0    3208
1.0    1593
Name: Case_Control, dtype: int64


In [22]:
columns=df.columns
print(columns)

Index(['Age', 'Gender', 'OccupationActivity',
       'TimeSittingPerDay_Weekend_UsingComputer', 'TypicalSleepHours_LastYear',
       'NapDuringDay_LastYear', 'Snoar', 'OtherHouseholdSmokers',
       'ParentsSmokedAtYourBirth', 'FatherAlive', 'HIPWAIST_OUT_WAIST_SIZE',
       'HIPWAIST_OUT_HIPS_SIZE', 'HIPWAIST_OUT_CALC_WAIST_TO_HIP_RATIO',
       'BP_OUT_ARM', 'BP_OUT_SYSTOLIC_BP_2', 'BP_OUT_PULSE_RATE_2',
       'BP_OUT_MANUAL_ENRTY_YN_2', 'BP_OUT_PULSE_RATE_1',
       'BP_OUT_ARMBAND_SIZE_USED', 'BP_OUT_MANUAL_ENRTY_YN_1',
       'BP_OUT_SLEEVE_TYPE', 'BP_OUT_DIASTOLIC_BP_2', 'BP_OUT_DIASTOLIC_BP_1',
       'BP_OUT_SYSTOLIC_BP_1', 'BP_OUT_CALC_AVG_SYSTOLIC_BP',
       'BP_OUT_CALC_AVG_DIASTOLIC_BP', 'BP_OUT_CALC_AVG_PULSE_RATE', 'BMI',
       'Case_Control'],
      dtype='object')


### Train with no augmentation

In [23]:
results={}

In [24]:
smote_results={}
adasyn_results={}

In [ ]:
results["0"]={}
smote_results["0"]={}
adasyn_results["0"]={}
source=train[train.columns[:-1]]
print(source.shape)


weight_path="weights/wt1.hdf5"
model,callbacks_list=create_model(weight_path,source.shape[1])
target = list(train["Case_Control"])
target=pd.get_dummies(target)
start=time.time()
print("Training MLP")
history = model.fit(source.values, target,epochs=600,validation_split=0.2,callbacks=callbacks_list,verbose=0)
end=time.time()
difference = int(end - start)
print("Time taken to train = ",difference)


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc='upper left')
plt.show()

# load weights
model.load_weights(weight_path)
# Compile model (required to make predictions)
opt=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

print("used model and loaded weights from file")

print("Trying MLP")
results["0"]["MLP"]={}
smote_results["0"]["MLP"]={}
adasyn_results["0"]["MLP"]={}

y_actual = list(test["Case_Control"])
y_actual=pd.get_dummies(y_actual)
test_features_only=test[test.columns[:-1]]
_, accuracy = model.evaluate(test_features_only.values, y_actual)
print('Accuracy: %.2f' % (accuracy*100))

y_actual = test["Case_Control"].astype(int)
y_actual=np.asarray(y_actual)
y_predict=model.predict_classes(test_features_only.values)
sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
results["0"]["MLP"]["F1"]=f1_score
results["0"]["MLP"]["sensitivity"]=sensitivity
results["0"]["MLP"]["specificity"]=specificity

smote_results["0"]["MLP"]["F1"]=f1_score  
smote_results["0"]["MLP"]["sensitivity"]=sensitivity
smote_results["0"]["MLP"]["specificity"]=specificity

adasyn_results["0"]["MLP"]["F1"]=f1_score  
adasyn_results["0"]["MLP"]["sensitivity"]=sensitivity
adasyn_results["0"]["MLP"]["specificity"]=specificity


# for other classifiers
target = list(train["Case_Control"])

# Random forest
print("RF")
results["0"]["RF"]={}
smote_results["0"]["RF"]={}
adasyn_results["0"]["RF"]={}

clf=RandomForestClassifier()
clf.fit(source.values,target)
y_pred=None
y_predict=clf.predict(test_features_only.values)
sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
results["0"]["RF"]["F1"]=f1_score
results["0"]["RF"]["sensitivity"]=sensitivity
results["0"]["RF"]["specificity"]=specificity

smote_results["0"]["RF"]["F1"]=f1_score  
smote_results["0"]["RF"]["sensitivity"]=sensitivity
smote_results["0"]["RF"]["specificity"]=specificity

adasyn_results["0"]["RF"]["F1"]=f1_score  
adasyn_results["0"]["RF"]["sensitivity"]=sensitivity
adasyn_results["0"]["RF"]["specificity"]=specificity

#     Gaussian NB
gnb = GaussianNB()
results["0"]["GNB"]={}
smote_results["0"]["GNB"]={}
adasyn_results["0"]["GNB"]={}

print("GNB")
gnb.fit(source.values,target)
y_predict = gnb.predict(test_features_only.values)
sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
results["0"]["GNB"]["F1"]=f1_score
results["0"]["GNB"]["sensitivity"]=sensitivity
results["0"]["GNB"]["specificity"]=specificity

smote_results["0"]["GNB"]["F1"]=f1_score  
smote_results["0"]["GNB"]["sensitivity"]=sensitivity
smote_results["0"]["GNB"]["specificity"]=specificity

adasyn_results["0"]["GNB"]["F1"]=f1_score  
adasyn_results["0"]["GNB"]["sensitivity"]=sensitivity
adasyn_results["0"]["GNB"]["specificity"]=specificity

#     SVM
clf = svm.SVC()
results["0"]["SVM"]={}
smote_results["0"]["SVM"]={}
adasyn_results["0"]["SVM"]={}

print("SVM")
clf.fit(source.values,target)
y_predict=clf.predict(test_features_only.values)
sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
results["0"]["SVM"]["F1"]=f1_score
results["0"]["SVM"]["sensitivity"]=sensitivity
results["0"]["SVM"]["specificity"]=specificity

smote_results["0"]["SVM"]["F1"]=f1_score  
smote_results["0"]["SVM"]["sensitivity"]=sensitivity
smote_results["0"]["SVM"]["specificity"]=specificity

adasyn_results["0"]["SVM"]["F1"]=f1_score  
adasyn_results["0"]["SVM"]["sensitivity"]=sensitivity
adasyn_results["0"]["SVM"]["specificity"]=specificity






In [ ]:
results

In [ ]:
smote_results

In [ ]:
adasyn_results

In [ ]:
train["Case_Control"].value_counts()

### Train  with augmentation

In [25]:
N_range=[500,1000,1500]
for N in N_range:
    results[N]={}
    smote_results[N]={}
    adasyn_results[N]={}
    
    k_range=[1,2,5,10]
    for k in k_range:
        results[N][k]={}
        smote_results[N][k]={}  
        adasyn_results[N][k]={}
        
        results[N][k]["MLP"]={}
        smote_results[N][k]["MLP"]={}  
        adasyn_results[N][k]["MLP"]={}

        results[N][k]["RF"]={}
        smote_results[N][k]["RF"]={}  
        adasyn_results[N][k]["RF"]={}
        
        results[N][k]["GNB"]={}
        smote_results[N][k]["GNB"]={}  
        adasyn_results[N][k]["GNB"]={}

        results[N][k]["SVM"]={}
        smote_results[N][k]["SVM"]={}  
        adasyn_results[N][k]["SVM"]={}


        class_index=1
        randmx=1
        dist_percent=0.2

        
        daug = data_augment.DataAugment()
        print("randmx = ",randmx)
        now = time.time()
        [Data_a,Ext_d,Ext_not]=daug.augment(data=train.values,k=k,class_ind=class_index,N=N,
                                            randmx=randmx,dist_percent=dist_percent)
        later = time.time()
        difference = int(later - now)
        print("Time taken to augment = ",difference)
        print(len(Data_a))

        train_aug=pd.DataFrame(data=Data_a,index=None,    # values                
                columns=columns)      

        print("After augmentation of ",N," items with ",k," neighbors")


        
        # this part for deep neural network
        
#         print(train_aug["Case_Control"].value_counts())
#         source=train_aug[train_aug.columns[:-1]]
#         target = list(train_aug["Case_Control"])
#         target=pd.get_dummies(target)

#         weight_path="weights/"+str(N)+"_"+str(k)+"_wt1.hdf5"    
#         model,callbacks_list=create_model(weight_path,source.shape[1])
#         start=time.time()
#         print("Training model for N = ",N," k = ",k)
#         history = model.fit(source.values, target,epochs=600,validation_split=0.2,callbacks=callbacks_list,verbose=0)
#         end=time.time()
#         difference = int(end - start)
#         print("Time taken to train = ",difference)



#         plt.plot(history.history['loss'])
#         plt.plot(history.history['val_loss'])
#         plt.title('model loss')
#         plt.ylabel('loss')
#         plt.xlabel('epoch')
#         plt.legend(['train','validation'], loc='upper left')
#         plt.show()    

#         # load weights
#         model.load_weights(weight_path)
#         # Compile model (required to make predictions)
#         opt=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#         model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#         print("used model and loaded weights from file")    
#         print("Test distribution")
#         print(test["Case_Control"].value_counts())
#         print("Trying MLP")
#         y_actual = list(test["Case_Control"])
#         y_actual=pd.get_dummies(y_actual)
#         test_features_only=test[test.columns[:-1]]
#         _, accuracy = model.evaluate(test_features_only.values, y_actual)
#         print('Accuracy: %.2f' % (accuracy*100))            


#         y_actual = test["Case_Control"].astype(int)
#         y_actual=np.asarray(y_actual)
#         y_predict=model.predict_classes(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         results[N][k]["MLP"]["F1"]=f1_score 
#         results[N][k]["MLP"]["sensitivity"]=sensitivity 
#         results[N][k]["MLP"]["specificity"]=specificity         
        


        # for other classifiers
        target = list(train_aug["Case_Control"])
        source=train_aug[train_aug.columns[:-1]]
        test_features_only=test[test.columns[:-1]]

        # Random forest
#         print("RF")
#         clf=RandomForestClassifier()
#         clf.fit(source.values,target)
#         y_predict=clf.predict(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         results[N][k]["RF"]["F1"]=f1_score 
#         results[N][k]["RF"]["sensitivity"]=sensitivity 
#         results[N][k]["RF"]["specificity"]=specificity         
                

    #     Gaussian NB
#         print("GNB")
#         gnb = GaussianNB()
#         gnb.fit(source.values,target)
#         y_predict = gnb.predict(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         results[N][k]["GNB"]["F1"]=f1_score 
#         results[N][k]["GNB"]["sensitivity"]=sensitivity 
#         results[N][k]["GNB"]["specificity"]=specificity         
                        
        

    #     SVM
        print("SVM")
        clf = svm.SVC()
        clf.fit(source.values,target)
        y_predict=clf.predict(test_features_only.values)
        sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
        results[N][k]["SVM"]["F1"]=f1_score 
        results[N][k]["SVM"]["sensitivity"]=sensitivity 
        results[N][k]["SVM"]["specificity"]=specificity         
                                
        
        
        
        # this part for SMOTE
        print("******Starting SMOTE***********")
        source=train[train.columns[:-1]]
        print("Shape of training ",source.shape)

        y_train=train["Case_Control"].values
        num_minority=sum(y_train==class_index)
        num_majority=len(y_train)-num_minority
        print(num_majority,num_minority)
        
        samp_strategy=(num_minority+N)/num_majority
        print(samp_strategy)
        now = time.time()
        sm = SMOTE(random_state=2,k_neighbors=k,sampling_strategy=samp_strategy)
        X_train_res, y_train_res = sm.fit_sample(source, y_train.ravel())
        source=X_train_res

        later = time.time()
        difference = int(later - now)
        print("Time taken to augment by SMOTE = ",difference)

        print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
        print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

        print("After OverSampling, counts of label {}: {}".format(class_index,sum(y_train_res==1)))
        print("After OverSampling, counts of other label: {}".format(len(y_train_res)-sum(y_train_res==class_index)))  
        

        # this pert for dnn
        
#         weight_path="weights/smote"+"_"+str(N)+"_"+str(k)+"_wt1.hdf5"    
#         model,callbacks_list=create_model(weight_path,X_train_res.shape[1])
#         start=time.time()
#         print("Training model")
#         target=pd.get_dummies(y_train_res)
#         history = model.fit(X_train_res.values, target,epochs=600,validation_split=0.2,callbacks=callbacks_list,verbose=0)
#         end=time.time()
#         difference = int(end - start)
#         print("Time taken to train = ",difference) 

#         plt.plot(history.history['loss'])
#         plt.plot(history.history['val_loss'])
#         plt.title('model loss')
#         plt.ylabel('loss')
#         plt.xlabel('epoch')
#         plt.legend(['train','validation'], loc='upper left')
#         plt.show()

        

#         # load weights
#         model.load_weights(weight_path)
#         # Compile model (required to make predictions)
#         opt=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#         model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#         print("used model and loaded weights from file")    
#         print("Trying MLP")
#         y_actual = list(test["Case_Control"])
#         y_actual=pd.get_dummies(y_actual)
#         test_features_only=test[test.columns[:-1]]
#         _, accuracy = model.evaluate(test_features_only.values, y_actual)
#         print('Accuracy: %.2f' % (accuracy*100)) 



#         y_actual = test["Case_Control"].astype(int)
#         y_actual=np.asarray(y_actual)
#         y_predict=model.predict_classes(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         smote_results[N][k]["MLP"]["F1"]=f1_score         
#         smote_results[N][k]["MLP"]["sensitivity"]=sensitivity                 
#         smote_results[N][k]["MLP"]["specificity"]=specificity                 

        # for other classifiers
        target = y_train_res
        source=X_train_res
        test_features_only=test[test.columns[:-1]]

        # Random forest
#         print("RF")
#         clf=RandomForestClassifier()
#         clf.fit(source.values,target)
#         y_predict=clf.predict(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         smote_results[N][k]["RF"]["F1"]=f1_score         
#         smote_results[N][k]["RF"]["sensitivity"]=sensitivity                 
#         smote_results[N][k]["RF"]["specificity"]=specificity

#     #     Gaussian NB
#         print("GNB")
#         gnb = GaussianNB()
#         gnb.fit(source.values,target)
#         y_predict = gnb.predict(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         smote_results[N][k]["GNB"]["F1"]=f1_score         
#         smote_results[N][k]["GNB"]["sensitivity"]=sensitivity                 
#         smote_results[N][k]["GNB"]["specificity"]=specificity

    #     SVM
        print("SVM")
        clf = svm.SVC()
        clf.fit(source.values,target)
        y_predict=clf.predict(test_features_only.values)
        sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
        smote_results[N][k]["SVM"]["F1"]=f1_score         
        smote_results[N][k]["SVM"]["sensitivity"]=sensitivity                 
        smote_results[N][k]["SVM"]["specificity"]=specificity

        
        
        
        
        # this part for ADASYN
        print("******Starting ADASYN***********")
        source=train[train.columns[:-1]]
        print("Shape of training ",source.shape)

        y_train=train["Case_Control"].values
        num_minority=sum(y_train==class_index)
        num_majority=len(y_train)-num_minority
        print(num_majority,num_minority)
        
        samp_strategy=(num_minority+N)/num_majority
        print(samp_strategy)
        now = time.time()
        ad = ADASYN(random_state=2,n_neighbors=k,sampling_strategy=samp_strategy)
        X_train_res, y_train_res = ad.fit_sample(source, y_train.ravel())
        source=X_train_res

        later = time.time()
        difference = int(later - now)
        print("Time taken to augment by ADASYN = ",difference)

        print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
        print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

        print("After OverSampling, counts of label {}: {}".format(class_index,sum(y_train_res==1)))
        print("After OverSampling, counts of other label: {}".format(len(y_train_res)-sum(y_train_res==class_index)))  
        
        
        # this part for DNN
        
#         weight_path="weights/adasyn"+"_"+str(N)+"_"+str(k)+"_wt1.hdf5"    
#         model,callbacks_list=create_model(weight_path,X_train_res.shape[1])
#         start=time.time()
#         print("Training model")
#         target=pd.get_dummies(y_train_res)
#         history = model.fit(X_train_res.values, target,epochs=600,validation_split=0.2,callbacks=callbacks_list,verbose=0)
#         end=time.time()
#         difference = int(end - start)
#         print("Time taken to train = ",difference) 

#         plt.plot(history.history['loss'])
#         plt.plot(history.history['val_loss'])
#         plt.title('model loss')
#         plt.ylabel('loss')
#         plt.xlabel('epoch')
#         plt.legend(['train','validation'], loc='upper left')
#         plt.show()

        

#         # load weights
#         model.load_weights(weight_path)
#         # Compile model (required to make predictions)
#         opt=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#         model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#         print("used model and loaded weights from file")    
#         print("Trying MLP")
#         y_actual = list(test["Case_Control"])
#         y_actual=pd.get_dummies(y_actual)
#         test_features_only=test[test.columns[:-1]]
#         _, accuracy = model.evaluate(test_features_only.values, y_actual)
#         print('Accuracy: %.2f' % (accuracy*100)) 



#         y_actual = test["Case_Control"].astype(int)
#         y_actual=np.asarray(y_actual)
#         y_predict=model.predict_classes(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         adasyn_results[N][k]["MLP"]["F1"]=f1_score         
#         adasyn_results[N][k]["MLP"]["sensitivity"]=sensitivity                 
#         adasyn_results[N][k]["MLP"]["specificity"]=specificity

        # for other classifiers
        target = y_train_res
        source=X_train_res
        test_features_only=test[test.columns[:-1]]

        # Random forest
#         print("RF")
#         clf=RandomForestClassifier()
#         clf.fit(source.values,target)
#         y_predict=clf.predict(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         adasyn_results[N][k]["RF"]["F1"]=f1_score         
#         adasyn_results[N][k]["RF"]["sensitivity"]=sensitivity                 
#         adasyn_results[N][k]["RF"]["specificity"]=specificity

#     #     Gaussian NB
#         print("GNB")
#         gnb = GaussianNB()
#         gnb.fit(source.values,target)
#         y_predict = gnb.predict(test_features_only.values)
#         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
#         adasyn_results[N][k]["GNB"]["F1"]=f1_score         
#         adasyn_results[N][k]["GNB"]["sensitivity"]=sensitivity                 
#         adasyn_results[N][k]["GNB"]["specificity"]=specificity

    #     SVM
        print("SVM")
        clf = svm.SVC()
        clf.fit(source.values,target)
        y_predict=clf.predict(test_features_only.values)
        sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)
        adasyn_results[N][k]["SVM"]["F1"]=f1_score         
        adasyn_results[N][k]["SVM"]["sensitivity"]=sensitivity                 
        adasyn_results[N][k]["SVM"]["specificity"]=specificity        
        
        
        



        


randmx =  1
Time taken to augment =  111
5301
After augmentation of  500  items with  1  neighbors
SVM


NameError: name 'y_actual' is not defined

In [ ]:
# get best

print("randmx = ",randmx)
print("dist_percent = ",dist_percent)
import math
checks=["MLP","RF","GNB","SVM"]
# checks=["GNB"]

best_n=None
best_k=None
best_sensitivity=0
best_specificity=0
best_f1=0

print("KNNOR")


for check in checks:
    for k,v in results.items():
        if k=="0":
            continue
        for k1,v1 in v.items():
#             print(k1)
#             print(v1)
            if math.isnan(v1[check]["F1"]):
                continue
#             print(v1[check]["F1"])
            if best_f1<v1[check]["F1"]:
                best_f1=v1[check]["F1"]
                best_n=k
                best_k=k1
                best_sensitivity=v1[check]["sensitivity"]
                best_specificity=v1[check]["specificity"]
                

    print(check,best_f1,"[N=",best_n,",k=",best_k,"]")
    print("Sensitivity = ",best_sensitivity)
    print("Specificity = ",best_specificity)    
    
    best_f1=0
    best_k=None
    best_n=None
    best_sensitivity=0
    best_specificity=0



In [ ]:
# get best

print("randmx = ",randmx)
print("dist_percent = ",dist_percent)
import math
checks=["MLP","RF","GNB","SVM"]
# checks=["GNB"]

best_n=None
best_k=None
best_sensitivity=0
best_specificity=0
best_f1=0

print("SMOTE")


for check in checks:
    for k,v in smote_results.items():
        if k=="0":
            continue
        for k1,v1 in v.items():
#             print(k1)
#             print(v1)
            if math.isnan(v1[check]["F1"]):
                continue
#             print(v1[check]["F1"])
            if best_f1<v1[check]["F1"]:
                best_f1=v1[check]["F1"]
                best_n=k
                best_k=k1
                best_sensitivity=v1[check]["sensitivity"]
                best_specificity=v1[check]["specificity"]                

    print(check,best_f1,"[N=",best_n,",k=",best_k,"]")
    print("Sensitivity = ",best_sensitivity)
    print("Specificity = ",best_specificity)
    
    best_f1=0
    best_k=None
    best_n=None
    best_sensitivity=0
    best_specificity=0
    



In [ ]:
# get best

print("randmx = ",randmx)
print("dist_percent = ",dist_percent)
import math
checks=["MLP","RF","GNB","SVM"]
# checks=["GNB"]

best_n=None
best_k=None
best_sensitivity=0
best_specificity=0
best_f1=0

print("ADASYN")


for check in checks:
    for k,v in adasyn_results.items():
        if k=="0":
            continue
        for k1,v1 in v.items():
#             print(k1)
#             print(v1)
            if math.isnan(v1[check]["F1"]):
                continue
#             print(v1[check]["F1"])
            if best_f1<v1[check]["F1"]:
                best_f1=v1[check]["F1"]
                best_n=k
                best_k=k1
                best_sensitivity=v1[check]["sensitivity"]
                best_specificity=v1[check]["specificity"]                

    print(check,best_f1,"[N=",best_n,",k=",best_k,"]")
    print("Sensitivity = ",best_sensitivity)
    print("Specificity = ",best_specificity)
    
    best_f1=0
    best_k=None
    best_n=None
    best_sensitivity=0
    best_specificity=0
    



### this part for trying with specific values

In [30]:
while True:
#     N_range=[500,1000,1500]
    N_range=[1500]
    for N in N_range:
        results[N]={}
        smote_results[N]={}
        adasyn_results[N]={}

    #     k_range=[1,2,5,10]
        k_range=[2]    
        for k in k_range:

#             randmx_range=[0.01,0.05,0.1,0.5,0.9]
            randmx_range=[0.12]

            for randmx in randmx_range:
#                 dist_percent_range=[0.01,0.05,0.1,0.5,0.9]
                dist_percent_range=[0.5]
                for dist_percent in dist_percent_range:



                    class_index=1
        #             randmx=0.12
                    dist_percent=0.5


                    daug = data_augment.DataAugment()
                    print("randmx = ",randmx)
                    print("dist_percent = ",dist_percent)        
                    now = time.time()
                    [Data_a,Ext_d,Ext_not]=daug.augment(data=train.values,k=k,class_ind=class_index,N=N,
                                                        randmx=randmx,dist_percent=dist_percent)
                    later = time.time()
                    difference = int(later - now)
            #         print("Time taken to augment = ",difference)
                    print(len(Data_a))

                    train_aug=pd.DataFrame(data=Data_a,index=None,    # values                
                            columns=columns)      

            #         print("After augmentation of ",N," items with ",k," neighbors")

            #         print(train_aug["Case_Control"].value_counts())
            #         source=train_aug[train_aug.columns[:-1]]
            #         target = list(train_aug["Case_Control"])
            #         target=pd.get_dummies(target)

            #         weight_path="weights/"+str(N)+"_"+str(k)+"_wt1.hdf5"    
            #         model,callbacks_list=create_model(weight_path,source.shape[1])
            #         start=time.time()
            #         print("Training model for N = ",N," k = ",k)
            #         history = model.fit(source.values, target,epochs=600,validation_split=0.2,callbacks=callbacks_list,verbose=0)
            #         end=time.time()
            #         difference = int(end - start)
            #         print("Time taken to train = ",difference)



            #         plt.plot(history.history['loss'])
            #         plt.plot(history.history['val_loss'])
            #         plt.title('model loss')
            #         plt.ylabel('loss')
            #         plt.xlabel('epoch')
            #         plt.legend(['train','validation'], loc='upper left')
            #         plt.show()    

            #         # load weights
            #         model.load_weights(weight_path)
            #         # Compile model (required to make predictions)
            #         opt=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
            #         model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

            #         print("used model and loaded weights from file")    
            #         print("Test distribution")
            #         print(test["Case_Control"].value_counts())
            #         print("Trying MLP")
            #         y_actual = list(test["Case_Control"])
            #         y_actual=pd.get_dummies(y_actual)
            #         test_features_only=test[test.columns[:-1]]
            #         _, accuracy = model.evaluate(test_features_only.values, y_actual)
            #         print('Accuracy: %.2f' % (accuracy*100))            


                    y_actual = test["Case_Control"].astype(int)
                    y_actual=np.asarray(y_actual)
            #         y_predict=model.predict_classes(test_features_only.values)
            #         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)


                    # for other classifiers
                    target = list(train_aug["Case_Control"])
                    source=train_aug[train_aug.columns[:-1]]
                    test_features_only=test[test.columns[:-1]]

                    # Random forest
            #         print("RF")
            #         clf=RandomForestClassifier()
            #         clf.fit(source.values,target)
            #         y_predict=clf.predict(test_features_only.values)
            #         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)


                #     Gaussian NB
            #         print("GNB")
            #         gnb = GaussianNB()
            #         gnb.fit(source.values,target)
            #         y_predict = gnb.predict(test_features_only.values)
            #         sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)



            #     #     SVM
                    print("SVM")
                    clf = svm.SVC()
                    clf.fit(source.values,target)
                    y_predict=clf.predict(test_features_only.values)
                    sensitivity,specificity,f1_score=check_cm_others(y_actual,y_predict)



                    if f1_score>.68:
                        print("*************************************")
                        print("N=",N)
                        print("k=",k)
                        print("randmx",randmx)
                        print("dist_percent",dist_percent)
                        print("f1",f1_score)
                        print("sensitivity",sensitivity)
                        print("specificity",specificity)




randmx =  0.12
dist_percent =  0.5
6301
SVM
Confusion Matrix : 
 [[576 216]
 [ 86 321]]
Accuracy :  0.7481234361968306
Sensitivity :  0.7886977886977887
Specificity :  0.7272727272727273
Precision =  0.5977653631284916
Recall =  0.7886977886977887
F1 score =  0.6800847457627118
*************************************
N= 1500
k= 2
randmx 0.12
dist_percent 0.5
f1 0.6800847457627118
sensitivity 0.7886977886977887
specificity 0.7272727272727273
randmx =  0.12
dist_percent =  0.5
6301
SVM
Confusion Matrix : 
 [[577 215]
 [ 86 321]]
Accuracy :  0.7489574645537949
Sensitivity :  0.7886977886977887
Specificity :  0.7285353535353535
Precision =  0.5988805970149254
Recall =  0.7886977886977887
F1 score =  0.6808059384941675
*************************************
N= 1500
k= 2
randmx 0.12
dist_percent 0.5
f1 0.6808059384941675
sensitivity 0.7886977886977887
specificity 0.7285353535353535
randmx =  0.12
dist_percent =  0.5
6301
SVM
Confusion Matrix : 
 [[576 216]
 [ 86 321]]
Accuracy :  0.748123436196

KeyboardInterrupt: 

In [ ]:
print("N=",N)
print("k=",k)
print("randmx",randmx)
print("dist_percent",dist_percent)
print("f1",f1_score)
print("sensitivity",sensitivity)
print("specificity",specificity)

